In [141]:
!git clone https://github.com/ngoxuanphong/ENV.git

Cloning into 'ENV'...
remote: Enumerating objects: 828, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 828 (delta 15), reused 28 (delta 4), pack-reused 774
Receiving objects: 100% (828/828), 50.22 MiB | 20.62 MiB/s, done.
Resolving deltas: 100% (325/325), done.


In [142]:
%cd ENV

/content/ENV/ENV/ENV/ENV/ENV/ENV/ENV/ENV/ENV/ENV/ENV/ENV/ENV/ENV


In [143]:

# Check hệ thống

# import Base.MachiKoro.env as env
# from CheckEnv import check_env
# print(check_env(env))

import numpy as np
import random as rd
from numba import njit, jit
import sys, os
from setup import SHORT_PATH
import importlib.util
game_name = 'Splendor'

In [144]:

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2


In [145]:
@njit()
def getGemsValue(state):
    gemsValue = np.zeros(5)
    myGems = state[201:206]
    myCards = state[207:212]
    cards = state[36:201].reshape(15,11)
    prices = np.zeros((15,5))
    # print(cards)
    for i in range(15):
        if np.sum(cards[i]) != 0:
            prices[i] = cards[i][6:11] - myGems - myCards
            for j in range(5):
              if prices[i][j] < 0: prices[i][j] = 0
              else: prices[i][j] = prices[i][j]**1.4
        else: prices[i] = np.full(5,99)
    # print(prices)
    cardsValue = np.zeros(15)
    for i in range(15):
        cardsValue[i] = (0.5*cards[i][0]+20.02)/(np.sum(prices[i]) +0.75)
    # print(cardsValue)
    idxmax = np.argmax(cardsValue)
    gemsValue = prices[idxmax]
    if np.sum(state[258:263]) == 1:
        gemsValue[np.where(state[258:263] == 1)[0][0]] = 0.0001
    for i in range(5):
        if gemsValue[i] == 0: gemsValue[i] = 0.01
    return gemsValue



In [146]:
@njit()
def StupidBot(state, per):
    ValidAction = getValidActions(state)
    ValidAction = np.where(ValidAction ==1)[0]

    cardCanBuy = np.zeros(90)
    for action in ValidAction:
        if action in range(5,95):
            cardCanBuy[action-5] = 1
    cardCanBuy = np.where(cardCanBuy ==1)[0]
    if len(cardCanBuy) > 0:
        action = cardCanBuy[np.random.randint(len(cardCanBuy))] + 5
        return action, per

    
    gems = np.zeros(5)
    for action in ValidAction:
        if action in range(5):
            gems[action] = 1
    # gems = np.where(gems ==1)[0] //bỏ
    if np.sum(gems) > 0:
        gemsValue = getGemsValue(state)
        # print(gemsValue)
        gemsValue = gemsValue*gems
        # print(gemsValue)

        action = np.argmax(gemsValue)
        # print(action, ValidAction,'-------------')
        return action, per
    action = ValidAction[np.random.randint(len(ValidAction))]
    return action, per

In [147]:
per =[np.zeros((1,1))]

In [150]:
numba_main_2(StupidBot,10000,per,1)

(7028, [array([[0.]])])